In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 1.查看自己是否有GPU，有就使用

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## 2.定义nn模型

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 对输入进行展开
        self.flatten = nn.Flatten()
        # 模型的层
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    # 前向传播过程
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# 模型实例化并转移到GPU上，这里如果没有GPU那么仍在CPU上
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
# 随机生成一个数据，并将其指定device
X = torch.rand(1, 28, 28, device=device)

# 前向传播
logits = model(X)

# 对结果进行softmax
pred_probab = nn.Softmax(dim=1)(logits)
# 取得其中最大值的索引当作类别
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


## 3.模型的Layers

In [7]:
# 随机生成一个矩阵（这里注意一下，以前仿照的是单通道的28，28的图片，现在是3通道，也可以视作3条）
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
# flatten的演示
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
# 线性层的演示
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
# relu激活的展示
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1020,  0.5016,  0.6345,  0.2780,  0.0573,  0.0249, -0.2846,  0.2273,
         -0.1942, -0.2875,  0.4988, -0.0968, -0.2809,  0.5053,  0.0411, -0.1028,
         -0.5755,  0.1940,  0.2414,  0.2868],
        [ 0.0041,  0.4441,  0.5011,  0.2754,  0.2976, -0.1346, -0.3806,  0.1028,
         -0.2003, -0.3094,  0.4199,  0.2170,  0.0357,  0.5989,  0.0412,  0.1393,
         -0.2759,  0.2999,  0.0496,  0.4288],
        [ 0.1191,  0.2174,  0.5183,  0.0805,  0.0457, -0.2870, -0.2059,  0.1480,
         -0.1642,  0.1174,  0.1038, -0.0619,  0.0348,  0.4548,  0.2127,  0.0063,
         -0.2326,  0.2660,  0.1505,  0.0909]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1020, 0.5016, 0.6345, 0.2780, 0.0573, 0.0249, 0.0000, 0.2273, 0.0000,
         0.0000, 0.4988, 0.0000, 0.0000, 0.5053, 0.0411, 0.0000, 0.0000, 0.1940,
         0.2414, 0.2868],
        [0.0041, 0.4441, 0.5011, 0.2754, 0.2976, 0.0000, 0.0000, 0.1028, 0.0000,
         0.0000, 0.4199, 0.2170, 0.0357, 0.5989, 0.041

In [12]:
# nn.Sequential的展示，串联
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits.shape

torch.Size([3, 10])

In [14]:
# softmax的展示
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab, pred_probab.sum(dim=1)

(tensor([[0.1111, 0.1340, 0.1066, 0.0781, 0.0942, 0.1108, 0.1008, 0.0702, 0.0875,
          0.1068],
         [0.1075, 0.1181, 0.1011, 0.0809, 0.0934, 0.1271, 0.0908, 0.0840, 0.1071,
          0.0900],
         [0.1204, 0.1230, 0.1048, 0.0875, 0.0837, 0.1365, 0.0821, 0.0715, 0.0878,
          0.1025]], grad_fn=<SoftmaxBackward>),
 tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>))

## 4.模型的参数

In [15]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0082, -0.0298, -0.0238,  ..., -0.0079,  0.0283,  0.0121],
        [ 0.0158,  0.0260, -0.0197,  ...,  0.0127, -0.0198,  0.0282]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0146, -0.0173], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0435, -0.0424,  0.0379,  ...,  0.0345, -0.0199,  0.0070],
        [ 0.0291, -0.0037, -0.0088,  ..., -0.0296, -0.0237,  0.0267]],
       device='cuda:0', grad_fn=<SliceBack